### Premissas:
- Nome: dim_seller
- Descrição da tabela: Tabela composta por variáveis qualitativas de vendedores.
- Tipo: SCD-2

In [0]:
# Importa bibliotecas
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, ArrayType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from datetime import datetime

In [0]:
%sql
SELECT DISTINCT
  seller_id,
  count(*)
FROM poc_datum.olist.sellers_silver 
GROUP BY seller_id
HAVING count(*) > 1

seller_id,count(1)


In [0]:
# Cria dataframe com dados para a criação da dimensão
df_dimensao = spark.sql("""
SELECT DISTINCT
  seller_id,
  seller_zip_code_prefix,
  seller_city,
  seller_state
FROM poc_datum.olist.sellers_silver 
""")

display(df_dimensao)

seller_id,seller_zip_code_prefix,seller_city,seller_state
f6122bc84774df1b372bdb3bb88ddb9f,6436,barueri,SP
8132b9bd16876e1b0f8808d43825dd48,88058,florianopolis,SC
a254c682cc01e119f83530446f1df9a9,16500,cafelandia,SP
cb9fb4ca75d7ba8437480e8dde64fe98,3551,sao paulo,SP
c5ebe6598748b0aeaa61cfb820478b92,95910,lajeado,RS
bd15ee794d5e640d9dd71b665b2ab15b,14078,ribeirao preto,SP
7994b065a7ffb14e71c6312cf87b9de2,29142,cariacica / es,ES
67883baaae6134ee81b271a542613728,1310,sao paulo,SP
f52c2422904463fdd7741f99045fecb6,9230,santo andre/sao paulo,SP
528bcf6680c36dddf07620bd35b33a6f,3178,sao paulo,SP


In [0]:
# Primeira carga

# Cria DataFrame com os dados de "NÃO INFORMADO"
data = [(-1, -1, "Não informado", "Não informado")]
column = df_dimensao.schema.fieldNames()
df = spark.createDataFrame( data, schema=column )
# df.show()

# Faz o join com o dataframe com dados para a criação da dimensão
dim_seller = df.union(df_dimensao)
# dim_customers.show()

# Inseri as colunas default de dimensão
dim_seller = dim_seller.withColumn( "row_ingestion_timestamp", current_timestamp() ) \
    .withColumn( "row_version", lit(1) ) \
    .withColumn( "row_current_indicator", lit(True) ) \
    .withColumn( "row_effective_date", to_timestamp( lit('1900-01-01 00:00:00'), "yyyy-MM-dd HH:mm:ss") ) \
    .withColumn( "row_expiration_date", to_timestamp( lit('2200-01-01 00:00:00') , "yyyy-MM-dd HH:mm:ss" ) )

# Inseri coluna SK
# dim_seller = dim_seller.withColumn( 'sk_dim_seller', monotonically_increasing_id() ) 
dim_seller = dim_seller.withColumn( 
    'sk_dim_seller', 
    sha2(concat_ws("|", 
        dim_seller.row_ingestion_timestamp,
        dim_seller.seller_id, 
        dim_seller.seller_zip_code_prefix), 256))

# Ordena as colunas
dim_seller_select = dim_seller.select( 
    'sk_dim_seller',
    'row_ingestion_timestamp',
    'row_version',
    'row_current_indicator',
    'row_effective_date',
    'row_expiration_date',
    'seller_id',
    'seller_zip_code_prefix',
    'seller_city',
    'seller_state'
    )

display(dim_seller_select)

sk_dim_seller,row_ingestion_timestamp,row_version,row_current_indicator,row_effective_date,row_expiration_date,seller_id,seller_zip_code_prefix,seller_city,seller_state
539b8e354c3f58a14f2a10bad3f123ca091f2b9f89ad2fde9626f014d286c253,2024-04-29T13:03:46.66Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,-1,-1,Não informado,Não informado
7d3aef7f670e3b9f186483ba400559d825b402544d98359ce9083c1c9ab5b7a4,2024-04-29T13:03:46.66Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,f6122bc84774df1b372bdb3bb88ddb9f,6436,barueri,SP
919dfcf54e094ae5c666d2c8f1c89cb3ffcfb8af60e980eac05f32e05f3cde02,2024-04-29T13:03:46.66Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,8132b9bd16876e1b0f8808d43825dd48,88058,florianopolis,SC
8e57c80765938bd2a69ff1d55c09cda2436be3c36de2b33ffa5e29d0bebbaa14,2024-04-29T13:03:46.66Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,a254c682cc01e119f83530446f1df9a9,16500,cafelandia,SP
ac637cc328862e45ae4339bce4550f18438552007c6c2a0b515fe528392382c8,2024-04-29T13:03:46.66Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,cb9fb4ca75d7ba8437480e8dde64fe98,3551,sao paulo,SP
ea4c30b31d1afcd50c95dc58996b925ba00d6dfbf89803794601b4b5c7c6ff1c,2024-04-29T13:03:46.66Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,c5ebe6598748b0aeaa61cfb820478b92,95910,lajeado,RS
7d7404f3ed9c78e7a680ab38daaf5e856de3084a703cb78d670a3a7a1c391ba3,2024-04-29T13:03:46.66Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,bd15ee794d5e640d9dd71b665b2ab15b,14078,ribeirao preto,SP
c7d450cda2d450435b2d78548fd2cc8af66f97f57b9c127b73f20c2acefe4b2b,2024-04-29T13:03:46.66Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,7994b065a7ffb14e71c6312cf87b9de2,29142,cariacica / es,ES
10b01cdfd9d05ea181154716f7e49280baedd229482a8014968000b47c3daf8f,2024-04-29T13:03:46.66Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,67883baaae6134ee81b271a542613728,1310,sao paulo,SP
32a88a8a17982af4c690f599500f678205e247e31bbf14d7f7e6c0077106fe20,2024-04-29T13:03:46.66Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,f52c2422904463fdd7741f99045fecb6,9230,santo andre/sao paulo,SP


In [0]:
table_name = 'dim_seller'
spark.sql('USE olist')
spark.sql(f'DROP TABLE IF EXISTS dim_seller')
dim_seller_select.write.format("delta").mode('overwrite').saveAsTable(table_name)

### Carga Diferencial (Upsert)

In [0]:
#teste
data = [('f6122bc84774df1b372bdb3bb88ddb9f',6436,'barueriL','SP')]
column = ['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']

df_dimensao = spark.createDataFrame(data, schema=column)
df_dimensao.show()

+--------------------+----------------------+-----------+------------+
|           seller_id|seller_zip_code_prefix|seller_city|seller_state|
+--------------------+----------------------+-----------+------------+
|f6122bc84774df1b3...|                  6436|   barueriL|          SP|
+--------------------+----------------------+-----------+------------+



In [0]:
# Dados Novos
df_origem = df_dimensao

# Dados da dimensão
df_destino = spark.sql("""
    SELECT   
        seller_id,
        seller_zip_code_prefix,
        seller_city,
        seller_state 
    FROM 
        poc_datum.olist.dim_seller
    ORDER BY seller_id """)

# Realize o EXCEPT (retornar apenas registros novos)
df_dados_novos = df_origem.exceptAll(df_destino)

display(df_dados_novos)

# Cria uma tabela temporária
df_dados_novos.createOrReplaceTempView("temp_dados_novos")

seller_id,seller_zip_code_prefix,seller_city,seller_state
f6122bc84774df1b372bdb3bb88ddb9f,6436,barueriL,SP


In [0]:
# Paramêtros
table_merge = 'dim_seller'
change_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
print(change_date)

2024-04-29 13:03:55.629720


In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMPORARY VIEW dados_novos AS 
SELECT
     sha2(concat_ws("|",  '{change_date}',  a.seller_id, a.seller_zip_code_prefix), 256) AS sk_dim_seller
    ,a.seller_id
    ,a.seller_zip_code_prefix
    ,a.seller_city
    ,a.seller_state
    ,to_timestamp('{change_date}')  AS change_date
    ,(
        SELECT
            MAX(b.sk_dim_seller)
        FROM
            dim_seller as b
        WHERE
            a.seller_id = b.seller_id
    ) AS max_sk_dim_seller
    ,COALESCE(
        (
            SELECT
                MAX(c.row_version) + 1
            FROM
                dim_seller as c
            WHERE
                a.seller_id = c.seller_id
        ), 1
    ) AS max_row_version
FROM
    temp_dados_novos AS a
""")

DataFrame[]

In [0]:
%sql
SELECT
    *
FROM
    dados_novos AS a

sk_dim_seller,seller_id,seller_zip_code_prefix,seller_city,seller_state,change_date,max_sk_dim_seller,max_row_version
5759692cf0bdb721dfe0edc4210c5f2bb96c87ffbff1a586999d89447ed74b0d,f6122bc84774df1b372bdb3bb88ddb9f,6436,barueriL,SP,2024-04-29T13:03:55.62972Z,a0216c7fb4e78acf44ec5a1420377d2f0ba96626d2874d8e684070855b7e2cb8,2


In [0]:
spark.sql(f""" 
MERGE INTO {table_merge} as destino
USING dados_novos 
ON destino.sk_dim_seller = dados_novos.max_sk_dim_seller

WHEN MATCHED THEN 
  UPDATE SET
   destino.row_expiration_date = to_timestamp('{change_date}') --dados_novos.change_date
  ,destino.row_current_indicator = False
  """)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql(f"""
MERGE INTO {table_merge} as destino
USING dados_novos 
ON destino.seller_id = dados_novos.seller_id
AND destino.seller_city = dados_novos.seller_city

WHEN NOT MATCHED 
  THEN INSERT (
    sk_dim_seller
    , row_ingestion_timestamp
    ,row_version
    ,row_current_indicator
    ,row_effective_date
    ,row_expiration_date
    ,seller_id
    ,seller_zip_code_prefix
    ,seller_city
    ,seller_state
  )
  VALUES (
    dados_novos.sk_dim_seller
    ,to_timestamp('{change_date}') --dados_novos.change_date
    ,dados_novos.max_row_version
    ,1
    ,to_timestamp('{change_date}') --dados_novos.change_date
    ,to_timestamp( '2200-01-01 00:00:00')
    ,dados_novos.seller_id
    ,dados_novos.seller_zip_code_prefix
    ,dados_novos.seller_city
    ,dados_novos.seller_state
  )
""")

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
SELECT * FROM poc_datum.olist.dim_seller
WHERE seller_id = 'f6122bc84774df1b372bdb3bb88ddb9f'

sk_dim_seller,row_ingestion_timestamp,row_version,row_current_indicator,row_effective_date,row_expiration_date,seller_id,seller_zip_code_prefix,seller_city,seller_state
5759692cf0bdb721dfe0edc4210c5f2bb96c87ffbff1a586999d89447ed74b0d,2024-04-29T13:03:55.62972Z,2,true,2024-04-29T13:03:55.62972Z,2200-01-01T00:00:00Z,f6122bc84774df1b372bdb3bb88ddb9f,6436,barueriL,SP
a0216c7fb4e78acf44ec5a1420377d2f0ba96626d2874d8e684070855b7e2cb8,2024-04-29T13:03:49.668Z,1,false,1900-01-01T00:00:00Z,2024-04-29T13:03:55.62972Z,f6122bc84774df1b372bdb3bb88ddb9f,6436,barueri,SP
